# **Import Libraries**

Throughtout this project we will use:

* **OpenCV**
* **Dlib**
* **NumPy**
* **Pillow** (will be used to help display images here in Jupyter Notebook)

In [2]:
import cv2
import dlib
import numpy as np
from PIL import Image

import glob
from IPython.display import display

# **Exctract facial features**<br>

<div align="center">
  <img src="https://pyimagesearch.com/wp-content/uploads/2017/04/facial_landmarks_68markup.jpg" width=400 height=300>
</div> <br>

With the help of **dlib**, we will extract the facial features of each image, as demonstraded in the picture above. We will use **get_frontal_face_detector** to detect where the face is, and **shape_predictor_68_face_landmarks.dat** to extract the 68 landmarks of the face:<br>

<div style="display: flex; justify-content: space-between; align-items: center;">
    <img src="https://cdn.discordapp.com/attachments/1027187677403029575/1332853023982092328/Screenshot_from_2025-01-25_23-09-41.png?ex=6796c39c&is=6795721c&hm=f73d98a5358d6dc37334a35abbb19fa9560d9d85cc26a8313599d659f5404ad2&" alt="Image 1" style="width: 50%; margin-right: 10px;" />
    <img src="https://cdn.discordapp.com/attachments/1027187677403029575/1332853066096971806/Screenshot_from_2025-01-25_23-10-20.png?ex=6796c3a6&is=67957226&hm=9fc02936c120d620f4c82fe383a3e2b1cf22d55e8bd7dcc0332be1dc84ec1d69&" alt="Image 3" style="width: 50%;" />
</div><br>

# **Normalize images**<br>

In drawing, faces can be simplified using the “Rule of Thirds,” which divides the face into horizontal and vertical thirds to help position facial features accurately.<br><br>

**Rule of Thirds:** The face can be divided into three equal horizontal sections. The top third is from the hairline to the eyebrows, the middle third is from the eyebrows to the bottom of the nose, and the bottom third is from the bottom of the nose to the chin. This method helps ensure that facial features are proportionally placed.<br>
**Facial Features Placement:** Eyes are typically positioned halfway between the top of the head and the chin, which aligns with the horizontal third division. The nose line is found in the middle of the eye line and the bottom of the chin, and the mouth line is about one-third of the way down from the nose line to the chin.<br><br>

Since each image has variant size, we need to normalize them, warping each to a **600x600 image**. With the drawing rules stated earlier, let's define where certain facial features will warp to:<br><br>

* Left corner of the **left eye**: (180, 200)
* Right corner of the **right eye**: (420, 200)
* **Bottom lip**: (300, 400)

Now that we know the starting and ending positions, we can use the similarity transform (rotation, translation and scale). To find this transformation, we will use **getAffineTransform**. With this matrix, we can warp the image with **warpAffine** and update the landmarks:<br>

<div align="center">
  <img src="https://learnopencv.com/wp-content/ql-cache/quicklatex.com-b6e614b5448854f2c83abcb6e5786774_l3.png" width=400 height=300>
</div> <br>

The *first* and *second* columns **rotate and scale** the vector. You will need you to add the *last* column, that represents the **translation**.

In [8]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

IMG_DIM = 600
LEFT_EYE_CORNER = 36
RIGHT_EYE_CORNER = 45
BOTTOM_LIP = 57


def detectFace(img):
    """
    Detect the face present in the image 
    
    Args:
      img (ndarray (width, height, 3)): Image
      
    Returns
        face (dlib_rectangles): Rectangle that delimits the face
    """    
    face = detector(img, 1)
    # if len(face) > 1 return largest
    # Couldn't detect any faces in the image
    if len(face) == 0:
        return None
    
    return face[0]


def normalizeImage(img, landmarks):
    """
    Warps the image input to a 600x600 image with the aid of the outer corner of the eyes and the bottom lip 
    
    Args:
      img (ndarray (width, height, 3)): Image
      landmarks (dlib_full_object_detection (68)): Facial landmarks of the image
      
    Returns
        (normalized_img, updated_landmarks) (ndarray (600, 600, 3), ndarray (68, 2)): Warped 600x600 image
                                                                                        and landmarks for the warped image
    """    
    left_eye = landmarks.part(LEFT_EYE_CORNER)
    right_eye = landmarks.part(RIGHT_EYE_CORNER)
    bot_lip = landmarks.part(BOTTOM_LIP)

    # Create source array with float32 type
    src = np.array([[left_eye.x, left_eye.y],
                    [right_eye.x, right_eye.y],
                    [bot_lip.x, bot_lip.y]], dtype=np.float32)
    
    # Create destination array with float32 type
    dst = np.array([[180, 200],
                    [420, 200],
                    [300, 400]], dtype=np.float32)

    # Get transformation matrix
    T = cv2.getAffineTransform(src, dst)

    # Warp Image
    normalized_img = cv2.warpAffine(img, T, (IMG_DIM,IMG_DIM))

    # Update landmarks (T*landmarks + translation), but first convert to ndarray
    npLandmarks = np.array([[landmarks.part(i).x, landmarks.part(i).y] for i in range(68)], dtype=np.float32)
    last_row = T[:, -1]
    translation = np.array([last_row for _ in range(68)], dtype=np.float32)
    
    updated_landmarks = (np.delete(T, -1, axis=1) @ npLandmarks.transpose()).transpose() + translation

    return (normalized_img, updated_landmarks.astype(int))

In [9]:
images_path = glob.glob("images/*")
normalized_imgs = []

for img_path in images_path:
    img = cv2.imread(img_path)
    face = detectFace(img)

    if face is None:
        print("Error: Couldn't detect any faces in:", img_path)
    else:
        print("Loading:", img_path)

        landmarks = predictor(img, face)
        normalized_img, landmarks = normalizeImage(img, landmarks)

        # Add more points to landmarks to help with alignment later on
        pass

        normalized_imgs.append((normalized_img, landmarks))

print("Done")

Loading: images/Donald_Trump.webp
Loading: images/Elon_Musk.webp
Loading: images/Mark_Zuckerberg.webp
Done


# **Align faces**

Now that the images are normalized, we need to allign the rest of the remaining features. To do that, we will use **Delaunay Triangulation**